In [73]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import ast
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
from pylab import rcParams

TRAIN = "Train"
EVAL = "Eval"

In [43]:
def embeddings_pca(embeddings_df, n_components=3):
    pca = PCA(n_components=n_components)
    pca.fit(embeddings_df)
    print('Components:', pca.components_)
    print('Explained variance', pca.explained_variance_)
    print('Explained variance ratio', pca.explained_variance_ratio_)
    embeddings = pca.transform(embeddings_df)
    return embeddings

In [84]:
def plot_corr(embeddings, labels, experiment_name):
    tasks = np.array([
        label[task_param]
        for label in labels
    ])
    tasks_embeddings = np.concatenate((embeddings, tasks), axis=1)
    columns = ["pca_{}".format(i) for i in range(3)]
    columns += ["task_{}".format(i) for i in range(3)]
    tasks_embeddings_df = pd.DataFrame(data=tasks_embeddings, columns=columns)
    ax = sn.heatmap(tasks_embeddings_df.corr(), annot=True)
    ax.set_ylim(0, 6)
    plt.title(experiment_name)
    plt.savefig('images/{}_corr.png'.format(experiment_name))
    plt.clf()

In [2]:
def plott(embeddings, labels, title='', task_param='velocity', show_labels=False, n_components=3):
    stages = np.array([
        TRAIN if label['stage'] == 'train' else EVAL 
        for label in labels
    ])
    tasks = np.array([
        float("{0:.2f}".format(
            #label[task_param][0]
            np.linalg.norm(np.array(label[task_param]) - np.array([0, 0.85, 0.175])) if type(label[task_param]) is list else label[task_param]
        ))
        for label in labels
    ])

    if n_components == 3:
        fig = px.scatter_3d(embeddings, x=0, y=1, z=2, color=tasks, symbol=stages)
    else:
        fig = px.scatter(embeddings, x=0, y=1, color=tasks, symbol=stages)

    fig.update_layout(
        title=title,
        legend=dict(x=-.1, y=-.1)
    )
    fig.show()

In [45]:
def plott2d(embeddings, labels, title='', task_param='velocity', show_labels=False, n_components=3, task_ax=0, pca_ax=0):
    stages = np.array([
        TRAIN if label['stage'] == 'train' else EVAL 
        for label in labels
    ])
    tasks = np.array([
        float("{0:.2f}".format(
            label[task_param][task_ax]
            #np.linalg.norm(np.array(label[task_param]) - np.array([0, 0.85, 0.175])) if type(label[task_param]) is list else label[task_param]
        ))
        for label in labels
    ])

    fig = px.scatter(embeddings, x=pca_ax, y=[0]*len(embeddings), color=tasks, symbol=stages)

    fig.update_layout(
        title='{} - Color: Task[{}], X: PCA[{}]'.format(title, task_ax, pca_ax),
        legend=dict(x=-.1, y=-.1)
    )
    #fig.show()
    fig.write_image("images/{}_task{}_pca{}.png".format(title, task_ax, pca_ax))

In [7]:
experiments = []

for dirname, _, _ in os.walk('output'):
    if 'embeddings' in dirname.split('/')[-1]:
        experiments.append(dirname)

experiments.sort()
for i, experiment in enumerate(experiments):
    print(i, experiment)

0 output/cheetah-vel/2020_02_16_00_49_55/embeddings
1 output/cheetah-vel/2020_02_17_18_09_55/embeddings
2 output/cheetah-vel/2020_03_03_16_20_26/embeddings
3 output/cheetah-vel/2020_03_04_17_23_41/embeddings
4 output/cheetah-vel/2020_03_05_23_41_01/embeddings
5 output/cheetah-vel/2020_03_06_13_23_33/embeddings
6 output/cheetah-vel/2020_03_07_23_12_39/embeddings
7 output/cheetah-vel/2020_03_17_00_33_59/embeddings
8 output/cheetah-vel/2020_03_26_04_30_37/embeddings
9 output/cheetah-vel/2020_03_26_04_34_42/embeddings
10 output/cheetah-vel/2020_03_27_00_51_58/embeddings
11 output/reach-ml1/2020_04_09_01_49_41-baseline-3adapt/embeddings
12 output/reach-ml1/2020_04_15_22_29_42-baseline/embeddings
13 output/reach-ml1/2020_04_18_23_23_57-baseline-ood/embeddings
14 output/reach-ml1/2020_04_18_23_25_34-rwscale/embeddings
15 output/reach-ml1/2020_04_21_15_12_49-l2-ood/embeddings
16 output/reach-ml1/2020_04_21_15_13_40-intobs-ood/embeddings
17 output/reach-ml1/2020_04_23_11_14_21-intctx-ood/embedd

In [8]:
results = [
    #(0, 'Int. Obs 1', 'velocity'),
    #(1, 'Int. Ctx 1', 'velocity'),
    #(2, 'PEARL 1', 'velocity'),
    #(3, 'PEARL 2', 'velocity'),
    #(4, 'Int. Ctx 2', 'velocity'),
    #(5, 'Ctx Only Grads', 'velocity'),
    #(6, 'Int. Ctx 3', 'velocity'),
    #(7, 'Int. Ctx 4', 'velocity'),
    #(8, 'L2 0.1', 'velocity'),
    #(9, 'Dropout', 'velocity'),    
    #(10, 'L2 0.01 1', 'velocity', '00002'),
    #(11, 'L2 0.01 2', 'velocity'),
    #(12, 'PEARL 3 (Small)', 'velocity'),
    #(3, 'Reach Normalized', 'goal')
    ('output/reach-ml1/2020_04_15_22_29_42-baseline/embeddings', 'PEARL', 'goal_pos', '00010'),
    #(13, 'PEARL OOD', 'goal_pos', '00010'),
    #(14, 'Scale', 'goal_pos', '00010'),
    #(15, 'L2 0.01', 'goal_pos', '00010'),
    #(16, 'Int Obs OOD', 'goal_pos', '00010'),
    #(17, 'Int Ctx OOD', 'goal_pos', '00010'), # curiosity (ctx) out of dist on reach
    ('output/reach-ml1/2020_04_23_13_48_31-intctx/embeddings', 'Int Ctx', 'goal_pos', '00010'),
    ('output/reach-ml1/2020_04_25_13_48_34-l2/embeddings', 'L2 0.01', 'goal_pos', '00010'),
    #$(20, 'L2 0.01 OOD', 'goal_pos', '00010'), # L2 reg on encoder out of dist on reach
]

In [85]:
mat = None
for experiment_path, experiment_name, task_param, adapt_index in results:
    path = os.path.join(experiment_path, adapt_index)
    embeddings_df = pd.read_csv(os.path.join(path, 'default/tensors.tsv'), sep='\t', header=None)
    labels_df = pd.read_csv(os.path.join(path, 'default/metadata.tsv'), sep='\t', header=None)

    embeddings = embeddings_pca(embeddings_df)
    labels = [ast.literal_eval(label.replace("array(", "").replace(", dtype=float32)", "")) for label in labels_df[0].array]
    plot_corr(embeddings, labels, experiment_name)

    for task_ax in range(3):
        pca_tasks = None
        for pca_ax in range(3):
            plott2d(embeddings, labels, title=experiment_name, task_param=task_param, show_labels=False, task_ax=task_ax, pca_ax=pca_ax)

Components: [[-0.24414917  0.09506505 -0.584323   -0.24569766  0.727704  ]
 [-0.1456495   0.96721772  0.20172689 -0.0426437  -0.02763844]
 [ 0.10027842 -0.06026502  0.13221682 -0.96786067 -0.17909999]]
Explained variance [2.99468788 0.80821656 0.6081793 ]
Explained variance ratio [0.61223592 0.16523232 0.12433657]
Components: [[ 0.98111415  0.14195567 -0.03669324 -0.11067091  0.06057366]
 [ 0.12348262 -0.09874998 -0.26265017  0.70519692 -0.6393064 ]
 [-0.01999897 -0.3416119  -0.63079597 -0.59991737 -0.35369045]]
Explained variance [2.72069513 0.64874588 0.3863943 ]
Explained variance ratio [0.66389429 0.15830465 0.09428656]
Components: [[ 0.01612984 -0.01103843 -0.99745077 -0.05973832 -0.03378269]
 [ 0.00103422 -0.04507441 -0.02592041 -0.11930039  0.99149522]
 [ 0.41340009 -0.88146223  0.00323001  0.2279153  -0.01299535]]
Explained variance [5.4585277  0.57507157 0.44890502]
Explained variance ratio [0.8143309  0.0857921  0.06696993]


<Figure size 720x720 with 0 Axes>